In [ ]:
# if mounting on google drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import pandas as pd
import random
from openai import OpenAI
OPEN_API_KEY = ""
client = OpenAI(api_key=OPEN_API_KEY)
import openai

In [ ]:
# insert names of fine-tuned models or outof the box models here
female_model = "ft:gpt-3.5-turbo-0125:personal::9KAqA7S4"
male_model = "ft:gpt-3.5-turbo-0125:personal::9KApTlQn"
neutral_model = "ft:gpt-3.5-turbo-0125:personal::9KApJkIv"
box = "gpt-3.5-turbo"

In [ ]:
filenames = ["promt2_template.txt"]
out_filename = ""
for filename in filenames:
  print(filename)
  f = open(filename ,mode='r', encoding='utf-8-sig')
  prompt = f.read()
  f.close()

  input = {"role": "user", "content": prompt}
  generated = []
  generations = pd.DataFrame(columns = ['Task', 'Output'])
  num = 0

  # generate at least 200 instnces
  while num < 200:
    completion = client.chat.completions.create(
      model=neutral_model,
      messages=[
        input
      ]
    )

    tasks = []
    outputs = []
    message = completion.choices[0].message.content.strip().split("\n\n")
    for i in message:

      # this checks that the output matches our expected format
      try:
        task, output = i.split("\n")
        task = task.replace("Task: ", "")
        output = output.replace("Output: ", "")
        new_df = pd.DataFrame([{"Task": task, "Output": output}])
        generations = pd.concat([generations, new_df], ignore_index=True)
        num = num + 1
      except:
        continue

    # modify input
    for i in message:
      if i not in generated:
        generated.append(message)

    prompt_split = prompt.split("\n\n")
    print(num)

    # replace 2 prompts
    """
    for i in range(2):
      rand_prompt = random.randint(1,9)
      rand_gen = random.randint(0, len(message) - 1)
      prompt_split = prompt.split("\n\n")
      prompt_split[rand_prompt] = message[rand_gen]
    prompt = "\n\n".join(prompt_split)
    input = input = {"role": "user", "content": prompt}
    name = filename.split(".")[0]
    """
  generations.to_csv(out_filename)

